In [ ]:
import numpy as np
import pandas as pd
#import os

#import plotly.graph_objects as go
#import plotly.express as px

from urllib.parse import urlparse
!pip install tldextract # this module is required to import tldextract
import tldextract
#from IPython.display import display
#from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

!pip install category_encoders # this module is required to import category_encoder
import category_encoders as ce

from sklearn.metrics import confusion_matrix, classification_report

import joblib



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path1 = '/content/drive/My Drive/Model/my_encoder.pkl'

In [ ]:
file_path2 = '/content/drive/My Drive/Model/my_scaler.pkl'

In [ ]:
file_path3 = '/content/drive/My Drive/Model/my_ensemble.pkl'

In [ ]:
import pandas as pd
from urllib.parse import urlparse
#this will be the input url
user_input = input("Enter a URL: ")
d={'url':[user_input]}
data=pd.DataFrame(d)

Enter a URL: http://cf.yp.yahoo.com/about/createfeedback


In [ ]:
data

,url
0,http://cf.yp.yahoo.com/about/createfeedback


In [ ]:
def extract_features(row):
    # Extract domain info using tldextract
    domain_info = tldextract.extract(row['url'])

    # Calculate scheme length
    scheme_len = len(urlparse(row['url']).scheme)

    # Calculate lengths of different URL components
    url_len = len(row['url'])
    path_len = len(urlparse(row['url']).path)
    param_len = len(urlparse(row['url']).params)
    query_len = len(urlparse(row['url']).query)
    frag_len = len(urlparse(row['url']).fragment)

    # Count specific symbols in the URL
    symbols = ['-', '@', '?', '%', '.']
    count_info = {}
    for sym in symbols:
        count_info['count' + sym] = row['url'].count(sym)

    # Count digits and alphabetic characters in the URL
    count_info['count_digit'] = sum(i.isdigit() for i in row['url'])
    count_info['count_alpha'] = sum(i.isalpha() for i in row['url'])

    # Combine all extracted features
    features = [domain_info.subdomain, domain_info.domain, domain_info.suffix,
                scheme_len, url_len, path_len, param_len, query_len, frag_len,
                count_info['count-'], count_info['count@'], count_info['count?'],
                count_info['count%'], count_info['count.'], count_info['count_digit'],
                count_info['count_alpha']]

    return features

def extract_url(data):
    # Apply the extract_features function to each row and expand the result into separate columns
    data[['subdomain', 'domain', 'suffix',
          'scheme_len', 'url_len', 'path_len', 'param_len', 'query_len', 'frag_len',
          'count-', 'count@', 'count?', 'count%', 'count.', 'count_digit', 'count_alpha']] = data.apply(extract_features, axis=1, result_type='expand')

    return data

In [ ]:
DataSet=extract_url(data)
DataSet

,url,subdomain,domain,suffix,scheme_len,url_len,path_len,param_len,query_len,frag_len,count-,count@,count?,count%,count.,count_digit,count_alpha
0,http://cf.yp.yahoo.com/about/createfeedback,cf.yp,yahoo,com,4,43,21,0,0,0,0,0,0,0,3,0,35


In [ ]:
cols = DataSet.columns

In [ ]:
count_encoder = joblib.load(file_path1)
category_scaler=joblib.load(file_path2)
DataSet.iloc[:, 1:4] = count_encoder.transform(DataSet.iloc[:, 1:4])
DataSet = pd.DataFrame(category_scaler.transform(DataSet.iloc[:, 1:]), columns=cols[1:])

<ipython-input-63-ecee47fb5f61>:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  DataSet.iloc[:, 1:4] = count_encoder.transform(DataSet.iloc[:, 1:4])


In [ ]:
DataSet

,subdomain,domain,suffix,scheme_len,url_len,path_len,param_len,query_len,frag_len,count-,count@,count?,count%,count.,count_digit,count_alpha
0,-1.094526,1.60192,1.296397,1.396959,0.098332,0.7702,-0.075979,-0.339948,2.49445,-0.364566,-0.074902,-0.553869,-0.188163,-0.31631,0.061077,0.197032


In [ ]:
ensemble_prediction= joblib.load(file_path3)

In [ ]:
final_answer=ensemble_prediction.predict(DataSet)

In [ ]:
print('label of the url :',final_answer.item())


label of the url : 1
